In [1]:
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
import datetime

## Step 1: Get Raw Json Data into Pandas Dataframe and drop unneccesary columns

In [2]:
senate_url = "https://senate-stock-watcher-data.s3-us-west-2.amazonaws.com/aggregate/all_transactions.json"
#house_url = "https://house-stock-watcher-data.s3-us-west-2.amazonaws.com/data/all_transactions.json"
data = pd.read_json(senate_url)
#house_data = pd.read_json(house_url)
data.drop(columns = ['ptr_link','asset_description','comment','disclosure_date'], inplace = True)

## Step 2: Save only asset_type == "Stock", change all "Exchange" to "Purchase", and get rid of empty tickers

In [3]:
# Save only Stocks
data = data[data["asset_type"] == "Stock"]
# Set "Exchange" to "Purchase"
for index,row in data.iterrows():
    if(data.at[index,'type'] == 'Exchange'):
        data.at[index,'type'] = 'Purchase'
# Get rid of rows with no ticker
no_ticker = data[data['ticker'] == '--']
data.drop(no_ticker.index,inplace=True)

data.head()

,transaction_date,owner,ticker,asset_type,type,amount,senator
1,10/26/2021,Joint,RMD,Stock,Sale (Full),"$1,001 - $15,000",Thomas H Tuberville
2,10/26/2021,Joint,APAM,Stock,Sale (Full),"$1,001 - $15,000",Thomas H Tuberville
3,10/26/2021,Joint,AON,Stock,Sale (Full),"$1,001 - $15,000",Thomas H Tuberville
9,10/18/2021,Joint,INTC,Stock,Purchase,"$50,001 - $100,000",Thomas H Tuberville
10,10/18/2021,Joint,AA,Stock,Sale (Full),"$100,001 - $250,000",Thomas H Tuberville


In [6]:
data.shape

(5805, 6)

## Step 3: Order Data by Senator,Ticker,Owner and sort those by time ascending

In [4]:
# Make data a tiered dataframe
data.set_index(["senator","ticker","owner"],inplace=True)
# Actually make transaction dates time objects
data['transaction_date'] = pd.to_datetime(data['transaction_date'])
# Sort dataframe
data.sort_values(by = ["senator","ticker","owner","transaction_date"],ascending=[True,True,True,True],inplace=True)


In [7]:
data.head()

transaction_date asset_type  \
senator                    ticker owner                                
A. Mitchell Mcconnell, Jr. IR     Spouse       2019-06-03      Stock   
                           VMC    Spouse       2019-06-03      Stock   
                           WFC    Spouse       2017-09-07      Stock   
                                  Spouse       2017-12-05      Stock   
                                  Spouse       2018-03-05      Stock   

                                                 type               amount  \
senator                    ticker owner                                      
A. Mitchell Mcconnell, Jr. IR     Spouse  Sale (Full)  $250,001 - $500,000   
                           VMC    Spouse  Sale (Full)  $250,001 - $500,000   
                           WFC    Spouse     Purchase     $1,001 - $15,000   
                                  Spouse     Purchase     $1,001 - $15,000   
                                  Spouse     Purchase     $1,001 - $15,000   

                                          basis  SPY basis  
senator                    ticker owner                     
A. Mitchell Mcconnell, Jr. IR     Spouse      0          0  
                           VMC    Spouse      0          0  
                           WFC    Spouse      0          0  
                                  Spouse      0          0  
                                  Spouse      0          0

## Step 4: Go through each row and use Ashley's Method to save stock and SPY Basis

In [5]:
data['basis'] = 0
data['SPY basis'] = 0

In [ ]:
start = -1
end = 8000
count = -1
for index,row in data.iterrows():
    #break;# We are done with this cell
    if(index < start):
        continue
    
    if(data.at[index,'basis'] != 0):
        continue
    ticker = row['ticker']#Ticker

    t_info = row['transaction_date'].split("/") # {09,30,2021}
    t_year = int(t_info[2])
    t_month = int(t_info[0])
    t_day = int(t_info[1])

    start_date = datetime.datetime(t_year,t_month,t_day)
    end_date = start_date + datetime.timedelta(days=7)#next weeks of start_dates
    

    #cast vars
    start_date = str(int(start_date.timestamp()))
    end_date = str(int(end_date.timestamp()))

    interval= '1d'
    #'1wk'
    #'1mo'
    #'1yr'
    event ='history'

    url = 'https://query1.finance.yahoo.com/v7/finance/download/'      + ticker + '?period1=' + start_date + '&period2=' + end_date + '&interval='      + interval + '&events=' + event
    spy_url = 'https://query1.finance.yahoo.com/v7/finance/download/'      + "SPY" + '?period1=' + start_date + '&period2=' + end_date + '&interval='      + interval + '&events=' + event
    try:
        b_data = pd.read_csv(url)
        spy_data = pd.read_csv(spy_url)
    except:
        data[index,'basis'] = -1 # lets us know website didn't work, as opposed to not reached in the search
        continue

    basis = (b_data["Open"][0] + b_data["Close"][0] + b_data["High"][0] + b_data["Low"][0]) / 4
    spy_basis = (spy_data["Open"][0] + spy_data["Close"][0] + spy_data["High"][0] + spy_data["Low"][0]) / 4
    
    data.at[index,'basis'] = basis
    data.at[index,'SPY basis'] = spy_basis

### That was a big step, so save data as csv

In [ ]:
data.to_csv("Final_Process_data1")

## Step 5: Make 2D list of dataframes holding each senators data

In [10]:
all_things=[]
last_ind = [] # neccesary since 2 rows for a sell and buy will have identical indecies, don't want to save a dataframe for both of them!
# ('Senator name',owner,ticker)
for index,row in data.iterrows():
    if(index == last_ind):
        continue
    last_ind = index
    temp = data[data.index == index]
    #if(temp.shape[0] > 1):
    all_things.append(temp)
len(all_things)


1944

In [14]:
all_things[8].head()

transaction_date asset_type         type  \
senator           ticker owner                                             
Angus S King, Jr. COP    Spouse       2017-11-28      Stock     Purchase   
                         Spouse       2020-08-18      Stock  Sale (Full)   

                                           amount  basis  SPY basis  
senator           ticker owner                                       
Angus S King, Jr. COP    Spouse  $1,001 - $15,000      0          0  
                         Spouse  $1,001 - $15,000      0          0

## Step 6: Combine the sales and purchases and append to new dataframe

#### Want to Create Dataframe with these columns<br>
["senator","ticker","owner","is_closed","p_date","p_basis","p_amount","p_SPY","s_date","s_basis","s_amount","s_SPY","percent_gain","SPY percent_gain"]

In [ ]:
for sen_data in all_things:
    # sen data is a dataframe, in date order with most recent date last
    all_purch = []
    all_sells = []
    purchase_ind = pd.Series({'A':[]})# We will store the rows of the data frame in here
    sell_ind = pd.Series({'A':[]})# And in here as well

    position_open = False # will be True when we see our first Purchase
    for index,row in sen_data.iterrows():
        # If this row is a purchase, app
        continue

### Step 7: From the big dataframe, run Ashley's method again for purchases with no sell(Open positions) to give a peak at the current holding